In [19]:
import matplotlib as mt
import pandas as pd
import pylab as pl
import numpy as np
import scipy as sc
import scipy.optimize as opt

data = pd.read_csv("raw.csv", header=(5))
data = data.convert_objects(convert_numeric=True)
data_c = pd.DataFrame()

## Prepare constant Values
RFPF = 0.0133
CH4_RF = 1.1
Tchiller = 7
Pamb = 98.0918852
Pchiller = Pamb + 23.75
xCO2intdry = 0.000375
xCO2dildry = 0.000375
xTHC_THC_FID_init = 1
print("Initialization finished!")

Initialization finished!


""


In [20]:
##### First Level of Calculation #####
data_c["E_CO2D2"] = data.E_CO2D2*10000 # % --> ppm
data_c["E_COHD2"] = data.E_COHD2*10000 # % --> ppm
data_c["E_NOXD2"] = data.E_NOXD2
data_c["E_THCW2"] = data.E_THCW2
data_c["xCH4wet"] = data.E_CH4W2/1000000 # ppm --> mol/mol
data_c["C_FRAIRWS"] = data.C_FRAIRWS
data_c["BARO Press"] = data.P_INLET*100 # in kPa
data_c["T_INLET"] = data.T_INLET
data_c["M_RELHUM"] = data.M_RELHUM
data_c["mfuel"] = data.M_FRFUEL*1000/3600 # kg/h --> g/s
print("First Level of Calculation finished!")

First Level of Calculation finished!


In [3]:
##### Second Level of Calculation #####
data_c["xCO2meas"] = data_c.E_CO2D2/1000000 # ppm --> mol/mol
data_c["xCOmeas"] = data_c.E_COHD2/1000000 # ppm --> mol/mol
data_c["xNOxmeas"] = data_c.E_NOXD2/1000000 # ppm --> mol/mol
data_c["xTHCmeas"] = data_c.E_THCW2/1000000 # ppm --> mol/mol
data_c["Molar Flow Wet"] = data_c.C_FRAIRWS*1000/3600 # kg/h --> g/s
data_c["maf_kgh (wet)"] = data_c.C_FRAIRWS
data_c["pH2O @ inlet"] = 10**(10.79574*(1-(273.16/(273.15+data_c.T_INLET)))-5.028*np.log10((273.15+data_c.T_INLET)/273.16)+0.000150475*(1-10**(-8.2969*(((273.15+data_c.T_INLET)/273.16)-1)))+0.00042873*(10**(4.76955*(1-(273.16/(273.15+data_c.T_INLET))))-1)-0.2138602)
data_c["xH2O"] = data_c.M_RELHUM*data_c.get("pH2O @ inlet")/(data_c.get("BARO Press")*100)
data_c["xTHC[THC_FID]cor"] = data_c.E_THCW2-xTHC_THC_FID_init
print("Second Level of Calculation finished!")

Second Level of Calculation finished!


In [4]:
##### Third Level of Calculation #####
data_c.E_THCW2 = data_c.get("xTHC[THC_FID]cor") # Corrected Value
data_c["xNO2meas"] = data_c.xNOxmeas*0
data_c["xNOmeas"] = data_c.xNOxmeas*1
data_c["xTHCwet"] = data_c.xTHCmeas
data_c["Intake Air flow"] = data_c.get("Molar Flow Wet")/28.96 # g/sec --> mol/sec
data_c["Mmix"] = 28.96559*(1-data_c.xH2O)+18.01528*data_c.xH2O
data_c["xH2Oint"] = data_c.xH2O
data_c["xH2Odil"] = data_c.xH2O

# Constant Channels
c = np.zeros((len(data_c),1))
c.fill(3.5)
data_c["xH2Ogas"] = c
c.fill(13.875389)
data_c["Molar Mass of HC Emissions"] = c
c.fill(28.0101)
data_c["Molar Mass of CO Emissions"] = c
c.fill(46.0055)
data_c["Molar Mass of NOx Emissions (as NO2)"] = c
c.fill(44.0095)
data_c["Molar Mass of CO2 Emissions"] = c
c.fill(13.875389)
data_c["Molar Mass of NMHC Emissions"] = c
c.fill(0.866)
data_c["Wc"] = c
print("Third Level of Calculation finished!")

Third Level of Calculation finished!


In [5]:
##### Fourth Level of Calculation #####
data_c["xNMHCwet"] = (data_c.xTHCwet-data_c.xCH4wet*CH4_RF)/(1-RFPF*CH4_RF)
data_c["nint (Intake Air Flow)"] = data_c.get("Molar Flow Wet")/data_c.Mmix
data_c["xH2Ointdry"] = data_c.xH2Oint/(1-data_c.xH2Oint)
data_c["xH2Odildry"] = data_c.xH2Odil/(1-data_c.xH2Odil)
print("Fourth Level of Calculation finished!")

Fourth Level of Calculation finished!


In [6]:
##### Fifth Level of Calculation #####
data_c["xCO2int"] = xCO2intdry/(1+data_c.xH2Ointdry)
data_c["xO2int"] = ((0.20982-xCO2intdry)/(1+data_c.xH2Ointdry)) 
data_c["xCO2dil"] = xCO2dildry/(1+data_c.xH2Odildry)
print("Fifth Level of Calculation finished!")

Fifth Level of Calculation finished!


In [16]:
##### Circle Calculation ##### 
def f(variables):
    # Channels in Formulas
    #A = data.xdil/exh -- G
    #B = data.xH2Oexh -- H
    #C = data.xCcombdry -- I
    #D = data.xH2Oexhdry -- J
    #E = data.xdil/exhdry -- K
    #F = data.xint/exhdry -- L
    #G = data.xraw/exhdry -- M
    #H = data.xH2OCOmeas -- AC
    #I = data.xH2OTHCmeas -- AD
    #J = data.xH2ONOxmeas -- AE
    #K = data.xH2ONO2meas -- AF
    #L = data.xH2OCO2meas -- AH
    #M = data.xCOdry -- AI
    #N = data.xTHCdry -- AJ
    #O = data.xNOxdry -- AK
    #P = data.xNO2dry -- AL
    #Q = data.xCO2dry -- AN
    #R = data.xH2dry -- AO    
    
    # Constant Values    
    alpha = 1.84
    beta = 0
    gamma = 0
    delta = 0
    h2ogas = 3.5    
    
    # Unknown Variables
    (A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R) = variables
        
    # Known Variables
    S = data_c.xCO2dil[i]
    T = data_c.xCO2int[i]
    U = alpha
    V = data_c.xH2Odil[i]
    W = data_c.xH2Oint[i]
    X = data_c.xO2int[i]
    Y = beta
    Z = gamma
    AA = delta
    AB = data_c.xCOmeas[i]
    AC = data_c.xTHCmeas[i]
    AD = data_c.xNOxmeas[i]
    AE = data_c.xNO2meas[i]
    AF = h2ogas
    AG = data_c.xCO2meas[i]
    
    # Equations to solve
    eq1 = 1-(G/(1+D))-A
    eq2 = D/(1+D)-B
    eq3 = Q+(M)+(N)-(S*E)-(T*F)-C
    eq4 = ((U/2)*(C-N))+V*E+W*F-R-D 
    eq5 = A/(1-B)-E
    eq6 = (1/(2*X))*(((U/2)-Y+2+(2*Z))*(C-N)-(M-O-(2*P)+R))-F
    eq7 = 0.5*(((U/2)+Y+AA)*(C-N)+((2*N)+M-P+R))+F-G
    eq8 = AB/(1-H)-M    
    eq10 = AC/(1-I)-N
    eq11 = B-I
    eq12 = AD/(1-J)-O    
    eq14 = AE/(1-K)-P    
    eq16 = (M*(D-V*E))/(AF*(Q-S*E))-R
    eq17 = AG/(1-L)-Q
    
    if a==mode:
        eq9 = B-H    
        eq13 = B-J
        eq15 = B-K
        eq18 = B-L
    else:
        eq9 = bs-H    
        eq13 = bs-J
        eq15 = bs-K
        eq18 = bs-L
    
    return [eq1,eq2,eq3,eq4,eq5,eq6,eq7,eq8,eq9,eq10,eq11,eq12,eq13,eq14,eq15,eq16,eq17,eq18]

i = 0
mode = 0
b = np.zeros((len(data_c),18))
bs = 0.00821722713286476 # Has to be calculated

while i < len(data_c)-1:
    i +=1  
    g = 0.5 # First guess for iteration start

    solution = opt.fsolve(f,(g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g))
    
    if solution[2]>bs:
        mode = 1
        solution = opt.fsolve(f,(g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g))
        mode = 0
    b[:][i] = solution

df = pd.DataFrame(b,columns =['xdil/exh','xH2Oexh','xCcombdry','xH2Oexhdry','xdil/exhdry','xint/exhdry','xraw/exhdry','xH2OCOmeas','xH2OTHCmeas','xH2ONOxmeas','xH2ONO2meas','xH2OCO2meas','xCOdry','xTHCdry','xNOxdry','xNO2dry','xCO2dry','xH2dry'])
data_c = pd.concat([data_c,df],axis=1)
print("Circle Calculation finished!")
## 05/06/2015 - Anton lang, Lang@FEV.com

Circle Calculation finished!


In [17]:
data_c


,E_CO2D2,E_COHD2,E_NOXD2,E_THCW2,xCH4wet,C_FRAIRWS,BARO Press,T_INLET,M_RELHUM,mfuel,...,xH2OTHCmeas,xH2ONOxmeas,xH2ONO2meas,xH2OCO2meas,xCOdry,xTHCdry,xNOxdry,xNO2dry,xCO2dry,xH2dry
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
1,328.48,6.92,-0.092079,111.581001,0.000003,2.240604,98.7028,21.630354,22.000578,0.037004,...,0.005723,0.005723,0.005723,0.005723,0.000007,0.000113,-9.260896e-08,-1.645233e-23,0.000330,0.000002
2,2499.44,7.95,-0.205617,112.227005,0.000003,35.378963,98.7166,21.597605,22.070593,0.536217,...,0.007721,0.007721,0.007721,0.007721,0.000008,0.000114,-2.072169e-07,-1.542569e-24,0.002519,0.000002
3,23072.99,483.56,3.720597,181.309998,0.000013,74.642776,98.7060,21.596458,22.110083,1.057249,...,0.026514,0.008217,0.008217,0.008217,0.000488,0.000187,3.751423e-06,-4.349587e-25,0.023264,0.000135
4,54156.50,4771.30,33.884899,2388.375000,0.000075,95.429771,98.7039,21.506634,22.139614,1.537282,...,0.055549,0.008217,0.008217,0.008217,0.004811,0.002530,3.416565e-05,4.231175e-23,0.054605,0.001396
5,79838.10,7574.71,35.785202,7476.959961,0.000119,97.734108,98.7144,21.625662,22.227383,1.563227,...,0.077919,0.008217,0.008217,0.008217,0.007637,0.008110,3.608169e-05,-2.856114e-23,0.080500,0.002226
6,102742.10,4755.15,21.162399,9948.900391,0.000084,94.523094,98.7071,21.562408,22.244352,1.546185,...,0.094234,0.008217,0.008217,0.008217,0.004795,0.010985,2.133774e-05,-1.440131e-22,0.103593,0.001354
7,113391.00,2985.46,18.444099,9970.299805,0.000051,90.488518,98.7020,21.583874,22.315147,1.533609,...,0.101405,0.008217,0.008217,0.008217,0.003010,0.011097,1.859691e-05,-5.469012e-23,0.114330,0.000839
8,120086.99,2298.79,18.670700,8184.040039,0.000036,89.470703,98.7078,21.596632,22.418831,1.501520,...,0.106091,0.008217,0.008217,0.008217,0.002318,0.009156,1.882539e-05,-5.989500e-24,0.121082,0.000643
9,125209.00,2037.67,25.428400,6696.949707,0.000029,86.723907,98.7197,21.627951,22.433193,1.481067,...,0.109768,0.008217,0.008217,0.008217,0.002055,0.007524,2.563908e-05,-3.362241e-24,0.126246,0.000569


In [15]:
##### Calculation of remaining values #####
data_c["xH2ONOmeas"] = data_c.xH2ONO2meas
data_c["xNOdry"] = data_c.xNOmeas/(1-data_c.xH2ONOmeas)
data_c["xCO2dry"] = data_c.xCO2meas/(1-data_c.xH2OCO2meas)
data_c["xH2dry"] = (data_c.xCOdry*(data_c.xH2Oexhdry-data_c.xH2Odil*data_c.get("xdil/exhdry")))/(data_c.xH2Ogas*(data_c.xCO2dry-data_c.xCO2dil*data_c.get("xdil/exhdry")))
data_c["xCOwet"] = data_c.xCOmeas*((1-data_c.xH2Oexh)/(1-data_c.xH2OCOmeas))
data_c['xNOxwet'] = data_c.xNOxmeas*((1-data_c.xH2Oexh)/(1-data_c.xH2ONOxmeas))
data_c["xNO2wet"] = data_c.xNO2meas*((1-data_c.xH2Oexh)/(1-data_c.xH2ONO2meas))
data_c["xNOwet"] = data_c.xNOmeas*((1-data_c.xH2Oexh)/(1-data_c.xH2ONOmeas))
data_c["xCO2wet"] = data_c.xCO2meas*((1-data_c.xH2Oexh)/(1-data_c.xH2OCO2meas))
data_c["xNOxcorrwet"] = data_c.xNOxwet*(18.84 *data_c.xH2O + 0.68094)

data_c["nexh"] = data_c.get("nint (Intake Air Flow)")/(1+((data_c.get("xint/exhdry")-data_c.get("xraw/exhdry"))/(1+data_c.xH2Oexhdry)))
data_c["Mass_THC"] = data_c.xTHCwet*data_c.nexh*data_c.get("Molar Mass of HC Emissions")
data_c["Mass_CO"] = data_c.xCOwet*data_c.nexh*data_c.get("Molar Mass of CO Emissions")
data_c["Mass_NOx"] = data_c.xNOxcorrwet*data_c.nexh*data_c.get("Molar Mass of NOx Emissions (as NO2)")
data_c["Mass_CO2"] = data_c.xCO2wet*data_c.nexh*data_c.get("Molar Mass of CO2 Emissions")
data_c["Mass_NMHC"] = data_c.xNMHCwet*data_c.nexh*data_c.get("Molar Mass of NMHC Emissions")

data_c.to_csv("Calculation_Report.csv")
print("Calculation finished and Report saved!")

Calculation finished!
Report saved.


In [13]:
# Emissions in total
sum_THC = data_c.Mass_THC.sum(skipna = True)
sum_CO = data_c.Mass_CO.sum(skipna = True)
sum_NOx = data_c.Mass_NOx.sum(skipna=True)
sum_CO2 = data_c.Mass_CO2.sum(skipna=True)
sum_NMHC = data_c.Mass_NMHC.sum(skipna=True)
sum_CO2


15175.740015659803